In [8]:
from utils.ModelWrapper import ModelWrapper
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import wandb

In [9]:
# Init
wandb_project_name = 'deepfake-basic'
data_dir = "./data/data-large/data"
img_size = 128
wandb.init(project=wandb_project_name, entity="dat550")

accuracy,▁▆█
epoch,▁▅█
loss,█▃▁
val_accuracy,▁▂█
val_loss,█▅▁
accuracy,0.66803
best_epoch,2
best_val_loss,0.60362
epoch,2
loss,0.60353
val_accuracy,0.66278


### Wandb Sweep training of the model

In [10]:
def train():
   # Default values for hyper-parameters we're going to sweep over
   config_defaults = {
         'epochs': 20,
         'batch_size': 32,
         'learning_rate': 0.0001,
         'optimizer': 'nadam',
         'hidden_layer_size': 64,
         'conv_layer_1_size': 16,
         'conv_layer_2_size': 32,
         'conv_layer_3_size': 64,
         'dropout': 0.5,
      }
   # Initialize wandb with a sample project name
   wandb.init(config=config_defaults, project=wandb_project_name, entity="dat550")

   # # Config is a variable that holds and saves hyperparameters and inputs
   config = wandb.config

   # init wrapper
   model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size)

   input_layers = [
      layers.Conv2D(filters = config.conv_layer_1_size, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)),
      layers.MaxPooling2D(),
      layers.Conv2D(filters = config.conv_layer_2_size, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(filters = config.conv_layer_3_size, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(config.dropout) if config.dropout > 0 else None,
      layers.Flatten(),
      layers.Dense(wandb.config.hidden_layer_size, activation='relu'),
      layers.Dense(units = 1, activation='sigmoid')
   ]
   input_layers = [e for e in input_layers if e is not None]

   # create model
   model_wrapper.create_model(None, input_layers, config)

   # Define the optimizer
   if config.optimizer=='sgd':
      optimizer = tf.keras.optimizers.SGD(lr=config.learning_rate) 
   elif config.optimizer=='rmsprop':
      optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate,)
   elif config.optimizer=='adam':
      optimizer = tf.keras.optimizers.Adam(lr=config.learning_rate,) 
   elif config.optimizer=='nadam':
      optimizer = tf.keras.optimizers.Nadam(lr=config.learning_rate,) 
   
   # Compile the model
   model_wrapper.model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
   # Train the model
   model_wrapper.fit()

In [ ]:
# sweep training
sweep_config = {
    "name": "deepfake-basic-sweep",
   'method': 'grid',
    "metric": {
            "name": "accuracy",
            "goal": "maximize"
        },
    "early_terminate": {
       "type": "hyperband",
       "min_iter": 3
   },
   'parameters': {
        'batch_size': {
            'values': [16, 32, 64] 
        },
        'dropout': {
            'values': [0.0, 0.2, 0.5]
        },
        # 'conv_layer_1_size': {
        #     'values': [16,32], #32
        # },
        # 'conv_layer_2_size': {
        #     'values': [32,64] #64
        # },
        # 'conv_layer_3_size': {
        #     'values': [64 ,128] # 128
        # },
        'hidden_layer_size': {
            'values': [64, 128, 256] # 256
        },
      'learning_rate': { #1e-2,
            'values': [ 1e-3, 1e-4, 3e-4]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'sigmoid', 'softmax']
        },
   }
}
sweep_id = wandb.sweep(sweep_config, project=wandb_project_name, entity="dat550")
wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: gfrul66h
Sweep URL: https://wandb.ai/dat550/deepfake-basic/sweeps/gfrul66h


wandb: Agent Starting Run: 4n8gsvid with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	dropout: 0
wandb: 	hidden_layer_size: 64
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.



wandb: ERROR Run 4n8gsvid errored: OSError(22, 'Klienten har ikke nødvendig tilgangsnivå')
wandb: Agent Starting Run: ahggzz8j with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	dropout: 0
wandb: 	hidden_layer_size: 64
wandb: 	learning_rate: 0.001
wandb: 	optimizer: nadam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.



### Evaluate model

In [ ]:
# load the bess model NB: edit namd and path
loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")
best_model = load_model(loaded_model.name)

In [ ]:
model_wrapper = ModelWrapper(data_dir, img_size, 32)
model_wrapper.evaluate_model(best_model)

Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.
  96/1587 [>.............................] - ETA: 4:22

KeyboardInterrupt: 

In [ ]:
model_wrapper.export_to_png()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
